In [21]:
import pandas as pd

train_data_frame = pd.read_csv("train.csv", encoding="utf-8")
test_data_frame = pd.read_csv("test.csv", encoding="utf-8")
train_median = train_data_frame.median()
train_data_frame = train_data_frame.fillna(train_median)
test_data_frame = test_data_frame.fillna(train_median)

C:\Users\s0u0b\AppData\Local\Temp\ipykernel_16372\3649915432.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_median = train_data_frame.median()


In [22]:
most_embarked = train_data_frame['Embarked'].value_counts().idxmax()
train_data_frame['Embarked'] = train_data_frame['Embarked'].fillna(most_embarked)
test_data_frame['Embarked'] = test_data_frame['Embarked'].fillna(most_embarked)
train_data_frame['Pclass_one_hot_encode'] = train_data_frame['Pclass']
test_data_frame['Pclass_one_hot_encode'] = test_data_frame['Pclass']

In [23]:
def get_tittle(full_name_with_tittle):
    keep_tittle = ["Mr", 'Miss', 'Mrs', 'Master', 'Dr', 'Rev']
    tittle = full_name_with_tittle.split(",")[-1].split(".")[0].strip()
    if tittle in keep_tittle:
        return tittle
    return None


train_data_frame['Name'] = train_data_frame['Name'].apply(get_tittle)
test_data_frame['Name'] = test_data_frame['Name'].apply(get_tittle)

In [24]:
from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()
train_data_frame["Sex"] = labelEncoder.fit_transform(train_data_frame["Sex"])
test_data_frame["Sex"] = labelEncoder.fit_transform(test_data_frame["Sex"])

In [25]:
train = pd.get_dummies(train_data_frame, columns=["Name", "Sex",'Embarked','Pclass_one_hot_encode'])
test = pd.get_dummies(test_data_frame, columns=["Name", "Sex",'Embarked','Pclass_one_hot_encode'])

In [26]:
train = train.drop(["PassengerId", "Ticket", "Cabin"], axis=1)
test = test.drop(["Ticket", "Cabin"], axis=1)

In [27]:
x_train = train.drop("Survived", axis=1)
y_train = train["Survived"]
x_test = test.drop("PassengerId", axis=1)
y_test = test["PassengerId"]

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

random_forest_classifier = RandomForestClassifier(n_estimators=36, max_depth=8)
scores = cross_val_score(random_forest_classifier, x_train, y_train, cv=10, n_jobs=4)
np.average(scores)

0.8272159800249688

In [29]:
random_forest_classifier = RandomForestClassifier(n_estimators=36, max_depth=8)
random_forest_classifier.fit(x_train, y_train)
pre = random_forest_classifier.predict(x_test)
result = pd.DataFrame({"PassengerId": y_test, "Survived": pre})
result.to_csv("submission.csv", encoding="utf-8", index=False)

In [30]:
key_list = list(x_train.columns)
value_list = random_forest_classifier.feature_importances_ * 100

dict_from_list = dict(zip(key_list, value_list))
dict_from_list

{'Pclass': 6.4695995566626845,
 'Age': 11.404044881659598,
 'SibSp': 5.154170191462713,
 'Parch': 2.9692385118795497,
 'Fare': 14.927877232767308,
 'Name_Dr': 0.21882277882161677,
 'Name_Master': 1.5617197234144402,
 'Name_Miss': 1.7806634085422979,
 'Name_Mr': 10.752844074858839,
 'Name_Mrs': 4.462572623072295,
 'Name_Rev': 0.23537684755733723,
 'Sex_0': 15.653136991916911,
 'Sex_1': 13.31129262238754,
 'Embarked_C': 1.2755101518581677,
 'Embarked_Q': 0.7364730264011109,
 'Embarked_S': 1.8090303788085422,
 'Pclass_one_hot_encode_1': 1.672385697748336,
 'Pclass_one_hot_encode_2': 1.110844217230238,
 'Pclass_one_hot_encode_3': 4.494397082950487}